# Pytorch Charformer Toxic Comment Classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install charformer-pytorch
!pip install transformers
!pip install pytorch-model-summary

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 17.9 MB/s 
     |████████████████████████████████| 596 kB 72.2 MB/s 
     |████████████████████████████████| 895 kB 78.7 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from charformer_pytorch import GBST
import torch.nn.functional as F
import re
import numpy as np
from transformers import T5Model, T5Config, BertConfig, BertModel
import tensorflow as tf

In [ ]:
!unzip /content/drive/MyDrive/processed.zip  -d /content/

Archive:  /content/drive/MyDrive/processed.zip
replace /content/processed/76.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: /content/processed/183.npy  
  inflating: /content/processed/29.npy  
  inflating: /content/processed/11.npy  
  inflating: /content/processed/13.npy  
  inflating: /content/processed/31.npy  
  inflating: /content/processed/174.npy  
  inflating: /content/processed/168.npy  
  inflating: /content/processed/182.npy  
  inflating: /content/processed/98.npy  
  inflating: /content/processed/178.npy  
  inflating: /content/processed/55.npy  
  inflating: /content/processed/59.npy  
  inflating: /content/processed/159.npy  
  inflating: /content/processed/157.npy  
  inflating: /content/processed/173.npy  
  inflating: /content/processed/7.npy  
  inflating: /content/processed/197.npy  
  inflating: /content/processed/50.npy  
  inflating: /content/processed/198.npy  
  inflating: /content/processed/8.npy  
  inflating: /content/processed/158.npy  
  inflat

In [ ]:
# from pytorch_model_summary import summary
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
# tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

# model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
# print(model)

In [ ]:
!unzip sample.zip

unzip:  cannot find or open sample.zip, sample.zip.zip or sample.zip.ZIP.


In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
class Salazzle(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = GBST(
                num_tokens = 257,             # number of tokens, should be 256 for byte encoding (+ 1 special token for padding in this example)
                dim = 512,                    # dimension of token and intra-block positional embedding
                max_block_size = 4,           # maximum block size
                downsample_factor = 4,        # the final downsample factor by which the sequence length will decrease by
                score_consensus_attn = True   # whether to do the cheap score consensus (aka attention) as in eq. 5 in the paper
                            )
        config = BertConfig(vocab_size=257, hidden_size=512, num_attention_heads=8)
        self.bert = BertModel(config=config,add_pooling_layer=True)
        self.regressor = torch.nn.Linear(512, 1)
        self.mask = torch.zeros(1, 1023).bool()
        self.loss = torch.nn.MSELoss()
        params = list(self.tokenizer.parameters()) + list(self.bert.parameters()) + list(self.regressor.parameters())
        self.optimizer = torch.optim.SGD(params, lr=0.00001, momentum=0.8)

    def forward(self,  data):
        #print(data.shape)
        #for row in data:
        x, self.mask = self.tokenizer(data, self.mask)
        #print(x)
        x = x.int()
        #print(x.shape) 
        x = self.bert(inputs_embeds=x,  attention_mask=self.mask)
        #print(x)
        return self.regressor(x[1])

    def train(self, data, labels, batch_size=20):

        xt, xv, yt, yv = train_test_split(data, labels, random_state=23, 
                                               test_size=.2, train_size=.8)
        tsteps = int(len(xt)/batch_size)
        vsteps = int(len(xv)/batch_size)
        #tsteps = int(len(data)/batch_size)
        #vsteps = int(len(data)/batch_size)
        ltotal = 0
        for i in tqdm(range(tsteps)):
            self.optimizer.zero_grad()
            self.mask = torch.zeros(1, 1023).bool()
            a, b = (i*batch_size, (i+1)*batch_size)
            x, y = (xt[a:b],yt[a:b])
            #x, y = (data[a:b],labels[a:b])
            pred = self.forward(x)
            #print(y)
            output = self.loss(pred, y)
            output.backward()
            self.optimizer.step()

            print(f'Loss: {output}')
        '''for i in tqdm(range(vsteps)):
            self.mask = torch.zeros(1, 1023).bool()
            a, b = (i*batch_size, (i+1)*batch_size)
            x, y = (xv[a:b],yv[a:b])
            pred = self.forward(x)
            output = self.loss(pred, y)
            ltotal += output
            print(f'Valid Total Loss: {ltotal}')'''

def preprocess(data, padding=1023):
    def b2(str):
        return [ord(x) for x in str]
    texts = []
    labels = []
    i = 0
    for row in data:
        text = re.sub('’', '\'', row[0])
        text = re.sub(r"[^a-zA-Z0-9\', *\u2019-]", '', text)
        #print(text)
        texts.append(b2(text))
        #print(i, b2(text))
        labels.append([float(row[-1])])
        i += 1
    #print(np.array(texts).shape)
    #print(max([max(row) for row in texts]))
    #print(min([min(row) for row in texts]))
    #print(type(labels[0]))
    texts = np.array([np.pad(x, (0, padding-len(x))) for x in texts], dtype=np.int32)
    #print(texts[840:845])

    return torch.tensor(texts), torch.tensor(labels)




        

In [ ]:
data = np.load('23.npy')
pokemodel = Salazzle()

FileNotFoundError: ignored

In [ ]:
texts, labels = preprocess(data)
#print(texts.shape)
pokemodel.train(texts, labels)

  0%|          | 1/400 [00:43<4:48:25, 43.37s/it]

Loss: 0.12115563452243805


  0%|          | 2/400 [01:24<4:37:10, 41.78s/it]

Loss: 0.11111140251159668


  1%|          | 3/400 [02:04<4:32:23, 41.17s/it]

...

Loss: 0.011083535850048065


100%|█████████▉| 398/400 [4:29:13<01:20, 40.47s/it]

Loss: 0.010617202147841454


100%|█████████▉| 399/400 [4:29:54<00:40, 40.49s/it]

Loss: 0.006979340221732855


# Testing

In [ ]:
!unzip /content/drive/MyDrive/processed.zip  -d /content/data/

In [ ]:
LENGTH = 1500

tokenizer = GBST(
                num_tokens = 257,             # number of tokens, should be 256 for byte encoding (+ 1 special token for padding in this example)
                dim = 512,                    # dimension of token and intra-block positional embedding
                max_block_size = 4,           # maximum block size
                downsample_factor = 4,        # the final downsample factor by which the sequence length will decrease by
                score_consensus_attn = True   # whether to do the cheap score consensus (aka attention) as in eq. 5 in the paper
                            )
content = np.load('/content/content/processed/processed/23.npy')
sentence = re.sub(r'[^a-zA-Z0-9\', *\u2019-]', '', content[0][0])
nums = np.array([ord(x) for x in sentence])[np.newaxis,...]
print(nums.shape)
num_pad = np.array([np.pad(x, (0, LENGTH-len(x))) for x in nums], dtype=np.uint8)
num_tensor = torch.Tensor(num_pad).int() #torch.tensor(nums, dtype=torch.int32).int()
mask = torch.ones(1, LENGTH).bool()
tokenizer(num_tensor, mask)

(1, 301)


(tensor([[[-0.2593,  0.2007, -0.4296,  ..., -0.4094, -0.1355,  0.1387],
          [-0.3552,  0.3135, -0.2614,  ..., -0.5194, -0.0745, -0.0489],
          [ 0.0355,  0.1879, -0.6218,  ..., -0.3660, -0.0185, -0.0304],
          ...,
          [-0.3119,  0.4016,  0.2220,  ..., -0.2196,  0.2812, -0.0251],
          [-0.3119,  0.4016,  0.2220,  ..., -0.2196,  0.2812, -0.0251],
          [-0.3962,  0.3343, -0.0088,  ..., -0.1387,  0.1720,  0.1018]]],
        grad_fn=<MaskedFillBackward0>),
 tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True, True,
          True, Tru

In [ ]:
print(num_tensor)

tensor([[ 72, 101,  32, 103, 111, 116,  32, 104, 105, 115,  32, 109, 111, 110,
         101, 121,  32, 110, 111, 119,  32, 104, 101,  32, 108, 105, 101, 115,
          32, 105, 110,  32, 119,  97, 105, 116,  32, 116, 105, 108, 108,  32,
          97, 102, 116, 101, 114,  32, 116, 104, 101,  32, 101, 108, 101,  99,
         116, 105, 111, 110,  32, 105, 110,  32,  50,  32, 121, 114, 115,  32,
         100, 105, 114, 116, 121,  32, 112, 111, 108, 105, 116, 105,  99, 105,
          97, 110, 115,  32, 110, 101, 101, 100,  32, 116, 111,  32,  98, 101,
          32,  97, 102, 114,  97, 105, 100,  32, 111, 102,  32,  84,  97, 114,
          32,  97, 110, 100,  32, 102, 101,  97, 116, 104, 101, 114, 115,  32,
          97, 103,  97, 105, 110,  32,  98, 117, 116,  32, 116, 104, 101, 121,
          32,  97, 114, 101, 110,  39, 116,  32,  97, 110, 100,  32, 115, 111,
          32, 116, 104, 101,  32, 112, 101, 111, 112, 108, 101,  32, 103, 101,
         116,  32, 115,  99, 114, 101, 119, 101, 100

In [ ]:
# Find max sentence length in file
max_len = 0
for i in range(1,201):
    content = np.load('/data/processed/' + str(i) + '.npy')
    for entry in content:
        sent = re.sub(r'[^a-zA-Z0-9\', *\u2019-]', '', entry[0])
        sent_len = len(sent)
        if sent_len > max_len:
            print(sent_len)
            print(sent)
            max_len = sent_len
            max_sent = sent

print(max_len)
print(max_sent)

In [ ]:
# Find max sentence length in all files
max_len = 0
for i in range(1,200):
    content = np.load('/data/processed/' + str(i) + '.npy')
    for entry in content:
        sent = re.sub(r'[^a-zA-Z0-9\', *\u2019-]', '', entry[0])
        sent_len = len(sent)
        if sent_len > max_len:
            print(f"FILE: {i}")
            print(sent_len)
            print(sent)
            max_len = sent_len
            max_sent = sent
        elif sent_len > 1023:
            print(f"FILE: {i}")
            print(sent_len)
            print(sent)

print(max_len)
print(max_sent)

# Old

In [ ]:
!tar xvzf sv-en.tgz

europarl-v7.sv-en.en
europarl-v7.sv-en.sv


In [ ]:
eng_full =""
swe_full = ""
with open("/content/europarl-v7.sv-en.en") as f:
    eng_full = f.readlines()
with open("/content/europarl-v7.sv-en.sv") as f:
    swe_full = f.readlines()

eng_pract = eng_full[0:25]
swe_pract = swe_full[0:25]

In [ ]:


#tokenizer = GBST(
#    num_tokens = 257,             # number of tokens, should be 256 for byte encoding (+ 1 special token for padding in this example)
#    dim = 512,                    # dimension of token and intra-block positional embedding
#    max_block_size = 4,           # maximum block size
#    downsample_factor = 4,        # the final downsample factor by which the sequence length will decrease by
#    score_consensus_attn = True   # whether to do the cheap score consensus (aka attention) as in eq. 5 in the paper
#)
#
#tokens = torch.randint(0, 257, (1, 1023)) # uneven number of tokens (1023)
#mask   = torch.ones(1, 1023).bool()
#
## both tokens and mask will be appropriately downsampled
#
#tokens, mask = tokenizer(tokens, mask = mask) # (1, 256, 512), (1, 256)
#
## now pass this on to your transformer

In [ ]:
def b(str):
    return bytes(str, 'utf-8')
def b2(str):
    return [ord(x) for x in str]

class EngSweModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.etokenizer = GBST(
                num_tokens = 257,             # number of tokens, should be 256 for byte encoding (+ 1 special token for padding in this example)
                dim = 512,                    # dimension of token and intra-block positional embedding
                max_block_size = 4,           # maximum block size
                downsample_factor = 4,        # the final downsample factor by which the sequence length will decrease by
                score_consensus_attn = True   # whether to do the cheap score consensus (aka attention) as in eq. 5 in the paper
                            )
        self.stokenizer = GBST(
                num_tokens = 257,             # number of tokens, should be 256 for byte encoding (+ 1 special token for padding in this example)
                dim = 512,                    # dimension of token and intra-block positional embedding
                max_block_size = 4,           # maximum block size
                downsample_factor = 4,        # the final downsample factor by which the sequence length will decrease by
                score_consensus_attn = True   # whether to do the cheap score consensus (aka attention) as in eq. 5 in the paper
                            )
        """tokens = torch.randint(0, 257, (1, 1023)) # uneven number of tokens (1023)
        mask   = torch.ones(1, 1023).bool()
        tokens, mask = tokenizer(tokens, mask = mask)"""
        # Initializing a BERT bert-base-uncased style configuration
        config_encoder = BertConfig(vocab_size=257, hidden_size=512,num_attention_heads=8)
        config_decoder = BertConfig(vocab_size=257, hidden_size=512,num_attention_heads=8)
        config_decoder.is_decoder = True
        config_decoder.add_cross_attention = True
        config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
        # Initializing a Bert2Bert model from the bert-base-uncased style configurations
        self.ed = EncoderDecoderModel(config=config)
        #model = EncoderDecoderModel.from_pretrained('my-model', config=encoder_decoder_config)
        #self.T5 = T5Model(T5Config(vocab_size=257))
        #self.ed = EncoderDecoderModel(encoder=self.T5.encoder,decoder=self.T5.decoder)
        self.emask = torch.ones(1, 1023).bool()
        self.smask = torch.ones(1, 1023).bool()

    def forward(self,  eng, swe):
        x, self.emask = self.etokenizer(eng, self.emask)
        y, self.smask = self.stokenizer(swe, self.smask)

        x = x.int()
        y = y.int()
        print(x.shape)


        return self.ed(inputs_embeds=x, attention_mask=self.emask, decoder_inputs_embeds=y, decoder_attention_mask=self.smask).last_hidden_state


In [ ]:
model = EngSweModel()
#print(model)

# TODO Iterator for batch size X seq_len val, train, and test sets
optim = torch.optim.SGD(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.StepLR(optim, 1, gamma=0.9)

# def train_model(eng,swe, model):
#     prediction = model.forward(eng,swe)
#     optim.zero_grad()
#     loss = F.cross_entropy(prediction, swe)
#     loss.backward()
#     optim.step()
def train_model(eng,swe, model):
    prediction = model.forward(eng,swe)
    optim.zero_grad()
    s, smask = model.stokenizer(swe, model.smask)
    loss = F.cross_entropy(prediction, s)
    loss.backward()
    optim.step()

def val_model(eng,swe, model):
    # TODO
    pass


"""eng = np.array()
swe = np.array([b2(y) for y in swe_pract])
print(eng[0])"""

# barely trust this
swe_int = [b2(y) for y in swe_pract]
#print(np.array(swe_int))
eng_int = [b2(y) for y in eng_pract]

max_lene = max([len(x) for x in eng_pract])
max_lens = max([len(x) for x in swe_pract])

even_len = 1023

swe_pad = np.array([np.pad(x, (0, even_len-len(x))) for x in swe_int], dtype=np.uint8)
#print(swe_pad.shape)
eng_pad = np.array([np.pad(x, (0, even_len-len(x))) for x in eng_int], dtype=np.uint8)


swe = torch.Tensor(swe_pad).int()
eng = torch.Tensor(eng_pad).int()


"""print(swe_set)
swe = torch.Tensor([b(y) for y in swe_pract])
eng = torch.Tensor([b(y) for y in eng_pract])
"""
print(eng.shape)
#print(eng[1200:2223].shape)
swe_ax = torch.Tensor(swe_pad[...,np.newaxis]).int()
eng_ax = torch.Tensor(eng_pad[...,np.newaxis]).int()
#print(swe_ax.shape)

for i in range (0,10):
    train_model(eng,swe, model)

prediction = model.forward(eng_pad[1200:2223], swe_pad[1200:2223])
print(prediction)
    


NameError: ignored

In [ ]:
print(len(eng_pract),len(swe_pract))

25 25


In [ ]:
ord('Å')

197

# Tensorflow Attempt

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install t5[gcp]
clear_output(wait=False)

In [ ]:
!pip install mesh-tensorflow
clear_output(wait=False)

In [ ]:
model = t5.from_pretrained()

In [ ]:
!unzip /content/transformer.zip -d /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer
clear_output(wait=False)

Archive:  /content/transformer.zip
  inflating: /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/cf_v2_d2_cv_base.gin  
  inflating: /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/cf_v2_d3_cv_base.gin  
  inflating: /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/charformer_layers.py  
  inflating: /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/custom_attention.py  
replace /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/transformer.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/transformer.py  


In [ ]:
!mv /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/gin/cf_v2_d2_cv_base.gin /usr/local/lib/python3.7/dist-packages/t5/models/gin/models
!mv /usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/gin/cf_v2_d3_cv_base.gin /usr/local/lib/python3.7/dist-packages/t5/models/gin/models

In [ ]:
import tensorflow as tf
import mesh_tensorflow as mtf
